# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_file = "city_data.csv"

city_df = pd.read_csv(city_data_file)
city_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,Seoul,1,KR,1602974574,93,37.57,126.98,6.00,0.40
1,1,Cape Town,20,ZA,1602974408,87,-33.93,18.42,15.00,3.10
2,2,East London,87,ZA,1602974684,70,-33.02,27.91,19.36,4.12
3,3,Rikitea,100,PF,1602974605,80,-23.12,-134.97,23.45,9.13
4,4,Thompson,90,CA,1602974684,92,55.74,-97.86,-1.00,2.10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Lat", "Long"]].astype(float)

# Convert Humidity to float and store
# Drop NaN values 
city_df_drop = city_df.dropna()
humidity = city_df["Humidity"].astype(float)

In [5]:
# Create a humidity Heatmap layer
fig_heat = gmaps.figure()

humidity_heat = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig_heat.add_layer(humidity_heat)

fig_heat

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# drop all na values 

# city_df_drop OR
city_df.dropna()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,Seoul,1,KR,1602974574,93,37.57,126.98,6.00,0.40
1,1,Cape Town,20,ZA,1602974408,87,-33.93,18.42,15.00,3.10
2,2,East London,87,ZA,1602974684,70,-33.02,27.91,19.36,4.12
3,3,Rikitea,100,PF,1602974605,80,-23.12,-134.97,23.45,9.13
4,4,Thompson,90,CA,1602974684,92,55.74,-97.86,-1.00,2.10
...,...,...,...,...,...,...,...,...,...,...
544,544,Gornopravdinsk,93,RU,1602974713,93,60.05,69.90,-1.20,6.53
545,545,Ouallam,62,NE,1602974713,26,14.32,2.09,33.61,4.28
546,546,Matara,86,LK,1602974713,85,5.95,80.54,25.24,5.08
547,547,Lakes Entrance,36,AU,1602974713,70,-37.88,147.98,14.44,0.89


In [17]:
# cities with Humidity with greater than 100
humindity_greater_than_100_city_df = city_df.loc[(city_df['Humidity'] > 100)]
humindity_greater_than_100_city_df

# There's none. Skipping this step...

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed


In [13]:
# clean up the data that has greater Humidity than 100. 
# cleaned_city_df = city_df_drop.loc[(city_df_drop['Humidity'] > 100)]
# cleaned_city_df      

# This is for reference guide:
# cleaned_city_df.loc[(cleaned_city_df['Max Temp'] >= 100) & (city_df_drop['Wind Speed'] >= 4)]
# cleaned_city_df_final = cleaned_city_df.loc[(cleaned_city_df['Max Temp'] >= 0) & (cleaned_city_df['Max Temp'] <= 10)]
# cleaned_city_df_final


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
# Store into variable named hotel_df.
hotel_df = pd.DataFrame(city_df)


# Add a "Hotel Name" blank column to the DataFrame.
hotel_df['Hotel Name'] = ""

In [19]:
# Retrieve new dataframe
hotel_df.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
0,0,Seoul,1,KR,1602974574,93,37.57,126.98,6.00,0.40,
1,1,Cape Town,20,ZA,1602974408,87,-33.93,18.42,15.00,3.10,
2,2,East London,87,ZA,1602974684,70,-33.02,27.91,19.36,4.12,
3,3,Rikitea,100,PF,1602974605,80,-23.12,-134.97,23.45,9.13,
4,4,Thompson,90,CA,1602974684,92,55.74,-97.86,-1.00,2.10,


In [20]:
# create an empty list of hotel_name
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "international hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Long"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
#         hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_name.append(name_address["results"][0]["name"])

    except (KeyError, IndexError):
        hotel_name.append(np.nan)

In [21]:
# drop any na/empty values
hotel_df.dropna().head(20)

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
0,0,Seoul,1,KR,1602974574,93,37.57,126.98,6.00,0.40,
1,1,Cape Town,20,ZA,1602974408,87,-33.93,18.42,15.00,3.10,
2,2,East London,87,ZA,1602974684,70,-33.02,27.91,19.36,4.12,
3,3,Rikitea,100,PF,1602974605,80,-23.12,-134.97,23.45,9.13,
4,4,Thompson,90,CA,1602974684,92,55.74,-97.86,-1.00,2.10,
5,5,Hithadhoo,52,MV,1602974684,77,-0.60,73.08,28.30,9.16,
6,6,Airai,57,TL,1602974684,65,-8.93,125.41,20.33,0.67,
7,7,Lavrentiya,76,RU,1602974507,87,65.58,-171.00,3.82,2.19,
8,8,Korla,16,CN,1602974684,52,41.76,86.15,7.45,0.63,
9,9,Mataura,89,NZ,1602974684,70,-46.19,168.86,10.56,2.68,


In [22]:
hotel_df.to_csv("hotel_data.csv", encoding="utf-8")

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [24]:
# Add marker layer ontop of heat map

# Convert hotel name to list
hotel_name_list = hotel_df["Hotel Name"].tolist()

# Create bank symbol layer
hotel_name_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel Names: {hotel}" for hotel in hotel_name_list]
)


fig = gmaps.figure()
fig.add_layer(hotel_name_list)

# Display figure
fig


TraitError: Element of the 'layers' trait of a Map instance must be a Widget, but a value of ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''